Install Dependencies

In [35]:
!pip install langchain langchain-openai langchain-core
!pip install langchain-community gradio

Data Loading, Chunking, and Embedding

In [52]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Example: load a web page (replace with your data source)
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Artificial_intelligence")
docs = loader.load()

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

LLM API Key Connenction And VectorDB

In [53]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from google.colab import userdata

# Initialize embedding model using Gemini
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=userdata.get('GOOGLE_API_KEY'))

# Create vector store and add chunks
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(chunks)

['98e7215f-8d4c-482c-8080-bd374646fee0',
 '29ee071b-7077-4d82-a2cf-0ca68aaeea32',
 'ecfb9efe-194b-4d79-9d72-1a8d24390c20',
 '0ee1cd71-59e4-4696-8704-b2597ac05f72',
 '0d0c0839-ddb3-4238-8d15-be4a9f63a6aa',
 '9e66d053-9882-4c2c-9882-65c93aa1387f',
 'ee2cf63c-dab5-4bc2-8f12-4ad3aec823f3',
 '2bcff230-eca3-45c0-b248-d59e965320ef',
 'd265f116-01ea-4409-aca8-53b71b4a9fe8',
 '68f2b22d-ee15-457a-9da9-ff576d4d3d98',
 '570831d3-3ade-40d4-879c-c4d623cdc464',
 '91fd9777-7c69-4939-a310-9b24341923ee',
 '16114bf2-589f-482b-9051-ec8675066611',
 'aa10921c-65f3-4664-acf2-6b55cbb27f65',
 '7873e4bf-e3b8-45bc-9c64-038488d8f391',
 '7aa45117-c822-4c8b-91f6-7415f5a6209f',
 '45ee7ebd-70f0-452f-a1b7-fd73f4321ae2',
 'a7ff78c7-bfd2-46c9-9ddb-0b439fb8fb81',
 '39676d39-1d5c-469c-aa41-0d8fd74d74bf',
 '890d06e2-b6c1-420e-9f2b-8560b562015c',
 '42e317f5-c032-4d10-97a7-f503d1f94b52',
 '380b870f-0f0a-48a4-a64d-924599ed00b8',
 '3e697446-ca0b-4e93-92db-5ab2d20f99bc',
 'd2188b68-5a0c-4f3d-a74e-2624bb1d0634',
 '57c3afac-2d01-

Input Query

In [54]:
query = "how autnomous ai agent works?"
retriever = vector_store.as_retriever()
relevant_docs = retriever.get_relevant_documents(query)

To use the Gemini API, add your API key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:

Output Generation

In [55]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

# Initialize chat model using Google Gemini
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", google_api_key=userdata.get('GOOGLE_API_KEY'))

# Compose prompt with retrieved context
context = "\n".join(doc.page_content for doc in relevant_docs)
prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

# Generate answer
response = llm.invoke(prompt)
print(response.content)

Autonomous AI agents work by perceiving their environment (through sensors or data inputs), using this perception to make decisions based on their programmed goals and algorithms, and then taking actions to achieve those goals.  This process is iterative: they perceive, decide, act, and then repeat the cycle.

Their autonomy stems from their ability to operate independently, without constant human intervention, within the limits of their programming and resources.  Many incorporate machine learning, allowing them to learn from experience and improve their decision-making and actions over time, adapting to new situations and optimizing their behavior.  However, they are still constrained by their programming, available computing power, and hardware limitations; they can only operate within their defined scope and have finite resources.


You can display the sources of the retrieved chunks to increase transparency

In [56]:
for doc in relevant_docs:
    print(f"Source: {doc.metadata.get('source', 'unknown')}")

Source: https://en.wikipedia.org/wiki/Artificial_intelligence
Source: https://en.wikipedia.org/wiki/Artificial_intelligence
Source: https://en.wikipedia.org/wiki/Artificial_intelligence
Source: https://en.wikipedia.org/wiki/Artificial_intelligence


In [57]:
def rag_qa(query):
    """
    Performs RAG to answer a query and includes sources.
    """
    # Retrieve relevant documents
    retriever = vector_store.as_retriever()
    relevant_docs = retriever.get_relevant_documents(query)

    # Compose prompt with retrieved context
    context = "\n".join(doc.page_content for doc in relevant_docs)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

    # Generate answer
    response = llm.invoke(prompt)
    answer = response.content

    # Add sources to the answer
    sources = "\nSources:\n" + "\n".join(f"- {doc.metadata.get('source', 'unknown')}" for doc in relevant_docs)

    return answer + sources

In [58]:
import gradio as gr
iface = gr.Interface(
    fn=rag_qa,
    inputs=gr.Textbox(label="Enter your question"),
    outputs=gr.Textbox(label="Answer with sources"),
    title="GEN-AI RAG Based Search Engine",
    description="Ask a question. The engine retrieves relevant info and generates a grounded answer with sources."
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e1bf9f71fab3daaba3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
